# Worksheet 5: Thresholds and climate extremes
The following exercises demonstrate analysis of moderate extremes in climate simulated in CORDEX. As with the other worksheets, these are just examples of some of the analysis that you might perform using packages such as Python and and the python Library IRIS.

The basis of climate extremes analysis is a common set of standard extreme climate indices, defined by the World Climate Research Programme [Expert Team on Climate Change Detection and Indices (ETCCDI)](https://www.wcrp-climate.org/etccdi).

There are 27 climate extremes indices, nicely summarised by the [Climdex](https://www.climdex.org/learn/indices/) website.  You can read more about them in the frame below...

In [ ]:
from IPython.display import IFrame
IFrame('https://www.climdex.org/learn/indices/', width=950, height=350)

In this worksheet we'll be looking at wet days, a threshold measure giving the count of days when $\mathrm{pr} \geq 1 mm \;day^{-1}$, and **R95p**, the 95th percentile of precipitation on wet days ($\mathrm{pr} \geq 1 mm \;day^{-1}$) in the 1986-2005 period.

<div class="alert alert-block alert-warning">
<b>By the end of this worksheet you should be able to:</b><br> 
- Have an appreciation for working with daily model data <br>
- Understand how to calculate some useful climate extremes statistics<br>
- Be aware of some coding stratagies for dealing with large data sets<br>
</div>

## Contents
### [5.1: Frequency of Wet days](#5.1) 
### [5.2: Calculating percentiles](#5.2)

## Preamble

In [ ]:
# Code preamble - these libraries will be used in this worksheet.
# This code block needs to be re-run every time you restart this worksheet!
%matplotlib inline 
import os
import iris
import iris.coord_categorisation
import iris.quickplot as qplt
import iris.plot as iplt
from iris.time import PartialDateTime
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np
import dask
dask.config.set(scheduler=dask.get)
import dask.array as da
from iris.analysis import Aggregator

# Some helpful data locations
DATADIR = 'data_v2'
DOMAIN = 'EAS-22'
CLIMDIR = os.path.join(DATADIR, DOMAIN, 'climatology')
HISTDIR = os.path.join(DATADIR, DOMAIN, 'historical')
FUTRDIR = os.path.join(DATADIR, DOMAIN, 'rcp85')
CHIRPSDIR = os.path.join('data_v2', 'CHIRPS')
GCMIDS = ['hadgem2-es', 'mpi-esm-lr']
TIME_PERIODS = {'historical':'1986_2005', 'rcp85':'2041_2060'}

---
<div class="alert alert-block alert-success">
    <b>Question:</b> Thinking about climate extremes, what model <b>averaging period</b> should we be using for our data analysis? Why? <br>
    How do we identify this model avergaing period in the model output <b>filenames</b>?
</div>

<b>Answer:</b><br>
*Type your answer here...*

---

<a id='5.1'></a>
## 5.1 Frequency of wet days

**a)** Start by finding the frequency of wet days using daily data for both _HadGEM2-ES_ and _MPI-ESM-LR_ driven runs.  Calculate the number of days in both the baseline and future periods which are wet days - **a wet day is defined as having precipitation >=1 mm/day**.  Then calculate the percentage of wet days.


In [ ]:
# For each day: is rainfall >= 1? True/False
# Sum over all days to get number of wet days at each grid point
# Do for both model simulations and time periods
# Then calcuate the percentage of wet days.

# Define a new aggregator to help count non-zero days
# (This uses a dask array to reduce memory load)
# To learn more about custom aggregators see
# https://scitools.org.uk/iris/docs/v2.4.0/examples/General/custom_aggregation.html#general-custom-aggregation 


for gcmid in GCMIDS:
    for period in TIME_PERIODS.keys():
        # Get path to daily data
        infile = os.path.join(DATADIR, DOMAIN, period, gcmid + '.day.' + TIME_PERIODS[period] + '.GERICS-REMO2015.pr.*.nc')
        data = iris.load_cube(infile)
        # count wet days 
        model_wetdays = data.collapsed('time', iris.analysis.COUNT,
                                       function=lambda values: values > 1)
        model_wetdays.rename(gcmid + ' number of wet days (>=1mm/day) ' + TIME_PERIODS[period])
        # Save the file
        outfile = os.path.join(CLIMDIR, gcmid + '.day.' + TIME_PERIODS[period] + '.GERICS-REMO2015.wetday.nc')
        iris.save(model_wetdays, outfile)
        print('Saved: ' + outfile)
        
        # Find wet days as a percentage of total days
        total_days = len(data.coord('time').points)  
        model_pcent_wetdays = (model_wetdays / total_days) * 100
        # Add metadata
        model_pcent_wetdays.rename(gcmid + ' percentage of wet days (>=1mm/day) ' + TIME_PERIODS[period])
        model_pcent_wetdays.units = '%'

        # Save the file
        outfile = os.path.join(CLIMDIR, gcmid + '.day.' + TIME_PERIODS[period] + '.GERICS-REMO2015.wetday.pcent.nc')
        iris.save(model_pcent_wetdays, outfile)
        print('Saved: ' + outfile)


**b) Calculate numbers of wet days and percentage of wet days from the _CHIRPS_ observations**. 

Climate Hazards Group InfraRed Precipitation with Station data ([CHIRPS](https://chc.ucsb.edu/data/chirps)) is a 35+ year quasi-global rainfall data set. Spanning 50°S-50°N (and all longitudes) and ranging from 1981 to near-present, CHIRPS incorporates climatology, 0.05° resolution satellite imagery, and in-situ station data to create gridded rainfall time series for trend analysis and seasonal drought monitoring. 

We'll use CHIRPS as our observational data from which to compare our CORDEX model data.

**Fill in the missing code** to calculate the observed wet days: 

In [ ]:
# Load CHIRPS daily precipitation data, but only period of interest
historical_time_constraint = iris.Constraint(time=lambda cell: PartialDateTime(year=1986) 
                            <= cell.point <= PartialDateTime(year=2005))

infile = os.path.join(CHIRPSDIR, 'chirps-v2.0_pr_day_1981-2018_p25.nc')

# Find number of wet days

# Save wet days cube
outfile = os.path.join(CLIMDIR, 'chirps.wetday.nc')

# Find number of days in dataset (number_chirps_days)

# Find wet days as percent of all chirps days 

# Save 
outfile = os.path.join(CLIMDIR, 'chirps.wetday.pcent.nc')


---
<div class="alert alert-block alert-success">
    <b>Question:</b> Are there any additional considerations that have to be made with daily data? <br>
    From a coding perspective, how does working with daily data compare to working with monthly data?
</div>

<b>Answer</b><br>
*Type your answer here...*

---

### Plotting

**c)** **Plot** the modelled and observed **numbers of wet days** from 1986-2005. 

In [ ]:
# Plot modelled and observed numbers of wet days for a common baseline period.
# Create a figure of the size desired
fig = plt.figure(figsize=(16, 4))
fig.suptitle('Number of wet days (1986-2005)', fontsize=16)

# Set common limits for each subplot
cbar_lims=(0,number_chirps_days)
easia_domain = [70,160,10,50]

# Load and plot the models' wet day count
for n, gcmid in enumerate(GCMIDS):
    infile = os.path.join(CLIMDIR, gcmid + '.day.1986_2005.GERICS-REMO2015.wetday.nc')
    nwetdays = iris.load_cube(infile)
    ax1 = fig.add_subplot(1, 3, n+1, projection=ccrs.PlateCarree())
    qplt.pcolormesh(nwetdays, vmin=cbar_lims[0], vmax=cbar_lims[1])
    plt.title(gcmid)
    ax1.coastlines()             # adds coastlines defined by the axes of the plot
    ax1.set_extent(easia_domain, crs=ccrs.PlateCarree())

# Load and plot CHIRPS wet day count
infile = os.path.join(CLIMDIR, 'chirps.wetday.nc')
obs_nwetdays = iris.load_cube(infile)
fig.add_subplot(1, 3, 3, projection=ccrs.PlateCarree())
qplt.pcolormesh(obs_nwetdays, vmin=cbar_lims[0], vmax=cbar_lims[1])
plt.title('Observations (CHIRPS)')
ax = plt.gca()              # gca function that returns the current axes
ax.coastlines()
ax.set_extent(easia_domain, crs=ccrs.PlateCarree())
plt.show()

---
<div class="alert alert-block alert-success">
<b>Question:</b> Which of the following steps are required in order to <b>calculate the model bias</b> (the difference between CORDEX model output and observed data)? <br>
    
* Regrid data onto a common grid, to the finer (higher) resolution <br>
* Regrid data onto a common grid, to the coarser (lower) resolution <br>
* Convert CODREX data to a regular lat-lon grid if the simulation used a rotated pole <br>
* Ensure the units are comparable (e.g not comparing K with C)  <br>
    
Which of these steps are required when __comparing output for different time periods from the same model simulation__ (e.g. _future - baseline_ difference calculations)? 
</div>    

<b>Answer</b><br>
Steps required to calculate the model bias: <br>
* *type your answer here...*<br>

Steps required when comparing output from the same simulation: <br>
* *type your answer here...*<br>


---

**d)** Calculate the **difference in modelled future and baseline** wet day frequency and also the **difference in modelled baseline and observation** wet day frequency.

In [ ]:
# Load percentage of wet days data for the CHIRPS observations
infile = os.path.join(CLIMDIR, 'chirps.wetday.pcent.nc')
obs = iris.load_cube(infile)
# Add coordinate system information to facilitate regridding later
wgs84_cs = iris.coord_systems.GeogCS(6371229.0) 
obs.coord('latitude').coord_system = wgs84_cs 
obs.coord('longitude').coord_system = wgs84_cs

# constrain the observations to a smaller domain 
obs_sub = obs.intersection(longitude=(70, 160), latitude=(10, 50))


# The observed rainfall data have been created using surface rain gauges, and so are only available
# over land points.  Define a mask to remove sea points. The mask is True for masked points.
mask = np.where(obs.data > 0.0, False, True)

# Redefine the obs data array as a masked array.
obs.data = np.ma.array(obs.data, mask=mask)

# Define regridding method
scheme = iris.analysis.Linear(extrapolation_mode='mask')

for gcmid in GCMIDS:
    infile = os.path.join(CLIMDIR, gcmid + '.day.' + TIME_PERIODS['historical'] + '.GERICS-REMO2015.wetday.pcent.nc')
    model_baseline = iris.load_cube(infile)
    infile = os.path.join(CLIMDIR, gcmid + '.day.' + TIME_PERIODS['rcp85'] + '.GERICS-REMO2015.wetday.pcent.nc')
    model_future = iris.load_cube(infile)
    # In order to compare the modelled and observed numbers of wet days,
    # the model data needs to be regridded to the CHIRPS grid
    model_baseline_rg = model_baseline.regrid(obs_sub, scheme)
    model_future_rg = model_future.regrid(obs_sub, scheme)

    # Find the difference between futue and baseline models
    diff_model = model_future_rg - model_baseline_rg
    diff_model.rename(gcmid +' change in number of wet days (>=1mm/day) 2041-2060 vs 1986-2005')
    # Save the file
    outfile = os.path.join(CLIMDIR, gcmid + '.GERICS-REMO2015.wetday.pcent.diff.nc')
    iris.save(diff_model, outfile)
    print('Saved ' + outfile)

    # Subtract the observed percentages of wet days from the modelled percentages
    diff_mod_obs = model_baseline_rg - obs_sub
    diff_mod_obs.rename(gcmid + ' number of wet days (>=1mm/day) bias compared to CHIRPS')
    # Save the file
    outfile = os.path.join(CLIMDIR, gcmid + '.GERICS-REMO2015.wetday.pcent.bias.nc')
    iris.save(diff_mod_obs, outfile)
    print('Saved ' + outfile)

<div class="alert alert-block alert-info">
<b>Note</b>: A cube can be easily constrained to a given domain using the <code>cube.intersection</code> method. More information on this (and other) Iris cube functionality can be found in the <b><a href="https://scitools.org.uk/iris/docs/v2.4.0/iris/iris/cube.html?highlight=intersection#iris.cube.Cube.intersection">Iris Documentation</a></b> online.
</div>

**e)** **Plot the percentage change and model bias for wet day frequency**.  

**First**, run the code block below to produce a series of plots showing the model bias and future precipitation change for _cahpa_ and _cahpb_ simulations. <br>

As the model domain is smaller than the observations domain, you will see that the data is plotted on a domain which is larger than necessary. 

**Next, read the Iris documentation** to learn how to use the `cube.intersection` method, then **add the necessary code** below, to constrain the plots to the model domain.


In [ ]:
# Create a figure of the size 12x12 inches
plt.figure(figsize=(12, 12))


# Load the model's future percentage change in wet days (future - baseline)
for n, gcmid in enumerate(GCMIDS):
    infile = os.path.join(CLIMDIR, gcmid + '.GERICS-REMO2015.wetday.pcent.diff.nc')
    pcent_change = iris.load_cube(infile)
    
    # Add in a line of code to constrain the model domain to these coordinates: 
    # longitude=(70, 160)
    # latitude=(10, 50)
    pcent_change_subset = 
    
    # plot percentage changes on first row
    plt.subplot(2, 2, n+1)
    qplt.pcolormesh(pcent_change_subset, 
                    vmax=10, vmin=-10, cmap='BrBG')
    plt.title(gcmid + ' rcp85-historical \n precipitation change (%)')
    ax = plt.gca()
    ax.coastlines()

    
# Load the percentage bias (differences in precipitation between the models and obs)
for n, gcmid in enumerate(GCMIDS):
    infile = os.path.join(CLIMDIR, gcmid + '.GERICS-REMO2015.wetday.pcent.bias.nc')
    pcent_bias = iris.load_cube(infile)
    
    # Add in a line of code to constrain the model domain to these lat-lon coordinates: 
    # longitude=(70, 160)
    # latitude=(10, 50)
    pcent_bias_subset = 
    
    # plot bias on the second row
    plt.subplot(2, 2, n+3)
    qplt.pcolormesh(pcent_bias_subset, 
                    vmax=20, vmin=-20, cmap='BrBG')
    plt.title(gcmid + ' model-observations \n precipitation difference (%)')
    ax = plt.gca()
    ax.coastlines()

plt.show()

---
<div class="alert alert-block alert-success">
    <b>Question:</b><br>

*  Which simulation (<i>HadGEM2-ES</i> or <i>MPI-ESM-LR</i>) has better agreement with observations for wet day frequency? <br>
* What is the magnitude and location of any notable wet or dry biases for each simulation during the baseline period? <br>
* Summarise the projected change in wet day frequency over South-East Asia for both simulations. <br>
* Given any wet/dry biases in each simulation's baseline period, what adjustments might we make to our summary of projected change from <i>HadGEM2-ES</i> or <i>MPI-ESM-LR</i>? 
    </div>

**Answers:**

Which simulation  has better agreement with observations?
    
* 

    
Magnitude and location of any notable wet or dry biases:

* HadGEM2-ES driven run: 
* MPI-ESM-LR driven run: 

Summarise the projected change in wet day frequency:

* HadGEM2-ES driven run: 
* MPI-ESM-LR driven run: 

Adjustments in light of model bias:

* HadGEM2-ES driven run: 
* MPI-ESM-LR driven run: 


---

<a id='5.2'></a>
## 5.2. Calculating percentiles

**f)** Calculate in mm/day the baseline (1986-2005) and future (2041-2060) **95th percentile of precipitation**. Do this for HadGEM2-ES, MPI-ESM-LE driven runs and also for CHIRPS  baseline.

This introduces some new processing challenges: **the size of the daily future data set is (probably) too large to load into memory**.  Sometimes Iris can handle this for us (see [Iris and Lazy Data](https://scitools.org.uk/iris/docs/v2.4.0/userguide/real_and_lazy_data.html)), but in this case we need to manually **'chunk' the data to load and process smaller sections**.  This way Iris only loads a section of the data at a time and keeps within the memory limits imposed by this computer.

In [ ]:
# Specify the number of chunks for the lat and lon dimensions
# This will give us 3 x 3 = 9 cubes
steps = (3, 3)  

# Define a helper function to extract our cube chunks
def chunks(cube, x, y):
    """Yield successive x-y sized chunks from cube"""
    for i in range(0, cube.coord(axis='x').shape[0], x):
        for j in range(0, cube.coord(axis='y').shape[0], y):
            yield cube[:, j:j + y, i:i + x]


Now process the data:

In [ ]:
# Loop over job ID and time period
for gcmid in GCMIDS:
    for period in TIME_PERIODS.keys():
        infile = os.path.join(DATADIR, DOMAIN, period, gcmid + '.day.' + TIME_PERIODS[period] + '.GERICS-REMO2015.pr.mmday-1.nc')
        model_precip = iris.load_cube(infile)
        # if cube has both latitude (2d true latitude) and grid_latitude, then
        # coord(axis='y') does not work, so remove unused 'latitude' and 'longitude'
        model_precip.remove_coord('latitude')
        model_precip.remove_coord('longitude')
        
        # Calculate lat-lon chunks in terms of their index
        lat_chunk = int(model_precip.coord(axis='y').shape[0] / steps[0])
        lon_chunk = int(model_precip.coord(axis='x').shape[0] / steps[1])
        # Make list of cubes
        subcubes = list(chunks(model_precip, lon_chunk, lat_chunk))
        # Loop through subcubes
        model_pc95 = iris.cube.CubeList()
        for cube in subcubes:
            model_pc95.append(cube.collapsed('time', iris.analysis.PERCENTILE, percent=95.))
        # Concatenate the cube list back into one cube
        model_pc95 = model_pc95.concatenate_cube()
        # Give cube a helpful name
        model_pc95.rename('R95p of ' + gcmid + ' daily rainfall ' + TIME_PERIODS[period])
        # Save output
        outfile = os.path.join(CLIMDIR, gcmid + '.day.pc95.' + TIME_PERIODS[period] + '.GERICS-REMO2015.pr.mmday-1.nc')
        iris.save(model_pc95, outfile)
        print('Saved: ' + outfile)
        # Tidy up memory
        del model_precip, model_pc95

---
<div class="alert alert-block alert-success">
    <b>Question:</b> Why can we only 'chunk' in the lat-lon dimensions?  Why can't we 'chunk' the time dimension?
</div>

<b>Answer</b><br>
<i>Type your answer here...</i>

---

Now do the same for the CHIRPS data. First we take a subset of the CHIRPS data to make the code run quicker - constraining the data in time to the same period as the RCM data above and in space to a similar area to the EAS-22 domain (62-185 degrees longitude, -2 to 50 degree latitude)

In [ ]:
historical_time_constraint = iris.Constraint(time=lambda cell: PartialDateTime(year=1986) 
                            <= cell.point <= PartialDateTime(year=2005))
# now load CHIRPS daily data
infile = os.path.join(CHIRPSDIR, 'chirps-v2.0_pr_day_1981-2018_p25.nc')
obs_precip = iris.load_cube(infile, historical_time_constraint)
obs_precip = obs_precip.intersection(longitude=(62, 185), latitude=(-2, 50))


# save this to disk 
subsetfile = os.path.join(CHIRPSDIR, 'chirps-v2.0_pr_day_1986-2005_p25_eastasia.nc')
iris.save(obs_precip, subsetfile)

# save memory
del obs_precip


In [ ]:
# Repeat the percentile calculation for the CHIRPS data

#  load subset of CHRIRPS in 
obs_precip = iris.load_cube(subsetfile)

# Calculate lat-lon chunks in terms of their index
lat_chunk = int(obs_precip.coord(axis='y').shape[0] / steps[0])
lon_chunk = int(obs_precip.coord(axis='x').shape[0] / steps[1])

# Make list of cubes
subcubes = list(chunks(obs_precip, lon_chunk, lat_chunk))
# Loop through subcubes
obs_pc95 = iris.cube.CubeList()
for cube in subcubes:
    cube.data = np.ma.filled(cube.data, np.nan)
    obs_pc95.append(cube.collapsed('time', iris.analysis.PERCENTILE, percent=95.))
    print('Done ' + len(obs_pc95) + ' chunks')

# Concatenate the cube list back into one cube
obs_pc95 = obs_pc95.concatenate_cube()

# Redefine the data array of pc95 as a masked array. 
obs_pc95.data = np.ma.masked_where(np.isnan(obs_pc95.data), obs_pc95.data)
outfile = os.path.join(CLIMDIR, 'chirps.pc95.1986_2005.mmday-1.nc')
iris.save(obs_pc95, outfile)
print('Saved: ' + outfile)

# Tidy up memory
del obs_precip, obs_pc95


**g)** **Calculate the change in extreme precipitation** _(the difference between the future and baseline 95th percentiles of precipitation)_ **and the associated model bias** _(the difference between the baseline and CHIRPS 95th percentiles of precipitation)._


In [ ]:
# Define WGS84 coordinate system
wgs84 = iris.coord_systems.GeogCS(semi_major_axis=6378137.0, inverse_flattening=298.257223563)

# Load CHIRPS data
infile = os.path.join(CLIMDIR, 'chirps.pc95.1986_2005.mmday-1.nc')
obs_cube = iris.load_cube(infile)
obs_cube.coord('latitude').coord_system = wgs84 
obs_cube.coord('longitude').coord_system = wgs84

# constrain the observations to a smaller domain 
obs_cube = obs_cube.intersection(longitude=(70, 160), latitude=(10, 50))


# Define regridding method
scheme = iris.analysis.Linear(extrapolation_mode='mask')

for gcmid in GCMIDS:
    # First, calculate the difference between the modelled future and baseline 95th percentiles
    infile = os.path.join(CLIMDIR, gcmid + '.day.pc95.1986_2005.GERICS-REMO2015.pr.mmday-1.nc')
    print(infile)
    model_base = iris.load_cube(infile)
    infile = os.path.join(CLIMDIR, gcmid + '.day.pc95.2041_2060.GERICS-REMO2015.pr.mmday-1.nc')
    model_fut = iris.load_cube(infile)
    diff = iris.analysis.maths.subtract(model_fut, model_base)
    diff.rename(gcmid + ' change in R95p (rcp85 - historical)')
    outfile = os.path.join(CLIMDIR, gcmid + '.day.pc95.diff.GERICS-REMO2015.pr.mmday-1.nc')
    iris.save(diff, outfile)
    print('Saved: ' + outfile)

    # Next, calculate the differences between the modelled baseline and observed 95th percentiles
    # Remember, to compare the model and observations, the model data need to be regridded.
    model_base_rg = model_base.regrid(obs_cube, scheme)
    bias = obs_cube.copy()
    bias.data = model_base_rg.data - obs_cube.data
    bias.rename(gcmid + ' bias in R95p (model - obs)')
    outfile = os.path.join(CLIMDIR, gcmid + '.day.pc95.bias.GERICS-REMO2015.pr.mmday-1.nc')
    iris.save(bias, outfile)
    print('Saved: ' + outfile)

**h)** **Plot the differences in the 95th percentiles** between the models and observations, and the future changes in the 95th percentiles of precipitation from both models.

**Complete the code block to plot the figures...**

<div class="alert alert-block alert-info">
<b>Note</b>: You will probably find it useful to consult the matpltlib documentation to help you produce your plots.  In this case, take a look at the <code>plt.subplot()</code> docs to help you arrange your plots:<b> <a href="https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.subplot.html"> plt.subplot()</a></b>.
</div>

In [ ]:
# HINT: The filenames have the following pattern: gcmid + '.day.pc95.bias.pr.mmday-1.nc'
# Create a figure of the size 12x12 inches
plt.figure(figsize=(12, 12))

for n, gcmid in enumerate(GCMIDS):
    # HINT: Use the `n` variable to help arrange you plots using: plt.subplot()
    # Use the matplotlib documention to help you! 

    # Load and plot the model bias (model - obs)

    # Load and plot the percentage change in precipitation between future and baseline

plt.show()

---
<div class="alert alert-block alert-success">
    <b>Question:</b><br>

* Where do we see the greatest changes in extreme precipitation for each simulation? <br>
* Comment on each model's ability to reprent observed extremes in precipitation at the 95th percentile. <br>
</div>

**Answer:**

Greatest changes:

* HadGEM2-ES driven run: 
* MPI-ESM-LR driven run: 

Abilty to represent observed extremes:

* HadGEM2-ES driven run: 
* MPI-ESM-LR driven run: 


---

<center>
<div class="alert alert-block alert-warning">
<b>This completes worksheet 5.</b> <br>You have calculated and compared climate indices for future and baseline rainfall. You have also learned an effective method for working with large quantities of daily data. <br>
In the final worksheet you will combine all the techniques learned to this point, through writing your own code to post-process and analyse CORDEX simulations of extreme temperature over East Asia. 
</div>
</center>

<p><img src="img/MO_MASTER_black_mono_for_light_backg_RBG.png" alt="python + iris logo" style="float: center; height: 100px;"/></p>
<center>© Crown Copyright 2022, Met Office</center>